# LOADING LIBRARIES

In [ ]:
%matplotlib inline
import os
import torch
import torchvision
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import math
import skimage.transform           # For resizing images
import skimage.morphology          # For using image labeling
import sklearn.model_selection     # For using KFold
import pandas as pd
import matplotlib.cm as cm         # Color map
import random

# NETWORK ARCHITECTURE

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batch_norm(x)
        x = F.relu(self.conv2(x))
        x = self.batch_norm(x)
        return x


class ReLayNet(nn.Module):
    def __init__(self):
        super(ReLayNet, self).__init__()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        
        self.drop1=nn.Dropout2d(p=0.3)
        self.drop2=nn.Dropout2d(p=0.5)
        self.drop3=nn.Dropout2d(p=0.5)

        self.up1 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.up2 = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.up3 = nn.MaxUnpool2d(kernel_size=2, stride=2)

        self.conv1 = ConvBlock(3, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)

        self.conv4 = ConvBlock(256, 256)

        self.conv5 = ConvBlock(512, 128)
        self.conv6 = ConvBlock(256, 64)
        self.conv7 = ConvBlock(128, 64)

        self.conv8 = nn.Conv2d(64, 3, 1)

    def forward(self, x):
        c1 = self.conv1(x)
        x, idx1 = self.pool1(c1)
        x = self.drop1(x)
        c2 = self.conv2(x)
        x, idx2 = self.pool2(c2)
        x = self.drop2(x)
        c3 = self.conv3(x)
        x, idx3 = self.pool3(c3)
        x = self.drop3(x)
        x = self.conv4(x)
        x = self.up1(x,idx3)
        x = torch.cat([x, c3], 1)
        x = self.conv5(x)
        x = self.up2(x,idx2)
        x = torch.cat([x, c2], 1)
        x = self.conv6(x)
        x = self.up3(x,idx1)
        x = torch.cat([x, c1], 1)
        x = self.conv7(x)
        x = self.conv8(x)
        return x

# LOADING NETWORK

In [ ]:
net=ReLayNet()
print(net)
net = net.cuda()

# LOADING DATASETS (.PTH FILES)

In [ ]:
'''
x=train image set
y=train image mask
z=train image mask inverse
v=train image boundaries
a=train weights
'''

In [ ]:
x=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TrSNew.pth")
y=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TrMNew.pth")
z=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TrMInvNew.pth")
v=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TrBNew.pth")
a=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TrWNew.pth")

In [ ]:
'''
u=3 channel target ([mask:mask inverse:boundary])
c=3 channel weight
'''

In [ ]:
w=torch.cat([y,z],1)
u=torch.cat([w,v],1)
b=torch.cat([a,a],1)
c=torch.cat([a,b],1)

In [ ]:
'''
x2=test image set
y2=test image mask
z2=test image mask inverse
v2=test image boundaries
a2=test weights
'''

In [ ]:
x2=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TsSNew.pth")
y2=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TsMNew.pth")
z2=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TsMInvNew.pth")
v2=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TsBNew.pth")
a2=torch.load("/home/histosr/Anuraag_Ranajoy/train2/TsWNew.pth")

In [ ]:
'''
u2=3 channel target ([mask:mask inverse:boundary])
c2=3 channel weight
'''

In [ ]:
w2=torch.cat([y2,z2],1)
u2=torch.cat([w2,v2],1)
b2=torch.cat([a2,a2],1)
c2=torch.cat([a2,b2],1)

# TRAINING NETWORK

In [ ]:
#defining hyperparameters, batch-size and no. of epochs
BatchSize=8
iterations = 20
lr=(4e-3)/0.85
decayrate=0.85

In [ ]:
trainloss = []
testLoss1 = []
testLoss2 = []
start = time.time()

count=0
for epoch in range(iterations):
    epochStart = time.time()
    runningloss = 0
    net.train(True)   #train start
    rn=torch.randperm(1382)
    x=x[rn]
    u=u[rn]
    c=c[rn]
    optimizer= optim.Adam(net.parameters(),lr=lr*decayrate*(count/4+1),betas=(0.9,0.99))
    for i in range(0,1380,BatchSize):
        inputs=x[i:i+BatchSize]
        #normalising inputs
        mean=torch.mean(torch.mean(inputs, dim=2),dim=2)
        for m in range(mean.shape[0]):
            for j in range(mean.shape[1]):
                s=torch.std(inputs[m][j],unbiased=False)
                inputs[m][j]=(inputs[m][j]-mean[m][j])/s
        labels=u[i:i+BatchSize]
        wt=c[i:i+BatchSize]
        
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        
        outputs = net(inputs)
        criterion=nn.BCEWithLogitsLoss(weight=wt.cuda())
        loss = criterion(F.softmax(outputs), labels/255.)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        runningloss += loss.data[0]
 
    avgTrainloss = runningloss/1380.0
    trainloss.append(avgTrainloss)
    
    net.train(False) # For testing
    test_runningLoss1 = 0    
    for i in range(0,188,BatchSize):
        
        inputs=x2[i:i+BatchSize]
        mean=torch.mean(torch.mean(inputs, dim=2),dim=2)
        for m in range(mean.shape[0]):
            for j in range(mean.shape[1]):
                s=torch.std(inputs[m][j],unbiased=False)
                inputs[m][j]=(inputs[m][j]-mean[m][j])/s
        labels=u2[i:i+BatchSize]
        wt=c2[i:i+BatchSize]
        inputs, labels = Variable(inputs.cuda()),Variable(labels.cuda())
        
        outputs = net(inputs)       
        
        criterion=nn.BCEWithLogitsLoss(weight=wt.cuda())
        
        loss = criterion(F.softmax(outputs), labels/255.)      
        
        test_runningLoss1 += loss.data[0] 
        
    avgTestLoss1 = test_runningLoss1/188.0    
    testLoss1.append(avgTestLoss1)
    
    count+=1
    
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainloss,'r--',label='train')        
    plt.plot(range(epoch+1),testLoss1,'g--',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
      
    epochEnd = time.time()-epochStart
    print('At Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f}; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTrainloss,epochEnd//60,epochEnd%60))
    print('At Iteration: {:.0f} /{:.0f}  ;  Testing Loss1: {:.6f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTestLoss1,epochEnd//60,epochEnd%60))
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))

# DEFINING METRIC FOR EVALUATION

In [ ]:
MIN_OBJECT_SIZE=15

def imshow_args(x):
    """Matplotlib imshow arguments for plotting."""
    if len(x.shape)==2: return x, cm.gray
    if x.shape[2]==1: return x[:,:,0], cm.gray
    return x, None

def get_labeled_mask(mask, cutoff=.5, min_object_size=MIN_OBJECT_SIZE):
    """Object segmentation by labeling the mask."""
    mask = mask.reshape(mask.shape[0], mask.shape[1])
    lab_mask = skimage.morphology.label(mask > cutoff) 
    
    # Keep only objects that are large enough.
    (mask_labels, mask_sizes) = np.unique(lab_mask, return_counts=True)
    if (mask_sizes < min_object_size).any():
        mask_labels = mask_labels[mask_sizes < min_object_size]
        for n in mask_labels:
            lab_mask[lab_mask == n] = 0
        lab_mask = skimage.morphology.label(lab_mask > .5) 
    
    return lab_mask

def get_iou(y_true_labeled, y_pred_labeled):
    """Compute non-zero intersections over unions."""
    # Array of different objects and occupied area.
    (true_labels, true_areas) = np.unique(y_true_labeled, return_counts=True)
    (pred_labels, pred_areas) = np.unique(y_pred_labeled, return_counts=True)

    # Number of different labels.
    n_true_labels = len(true_labels)
    n_pred_labels = len(pred_labels)

    # Each mask has at least one identified object.
    if (n_true_labels > 1) and (n_pred_labels > 1):
        
        # Compute all intersections between the objects.
        all_intersections = np.zeros((n_true_labels, n_pred_labels))
        for i in range(y_true_labeled.shape[0]):
            for j in range(y_true_labeled.shape[1]):
                m = y_true_labeled[i,j]
                n = y_pred_labeled[i,j]
                all_intersections[m,n] += 1 

        # Assign predicted to true background.
        assigned = [[0,0]]
        tmp = all_intersections.copy()
        tmp[0,:] = -1
        tmp[:,0] = -1

        # Assign predicted to true objects if they have any overlap.
        for i in range(1, np.min([n_true_labels, n_pred_labels])):
            mn = list(np.unravel_index(np.argmax(tmp), (n_true_labels, n_pred_labels)))
            if all_intersections[mn[0], mn[1]] > 0:
                assigned.append(mn)
            tmp[mn[0],:] = -1
            tmp[:,mn[1]] = -1
        assigned = np.array(assigned)

        # Intersections over unions.
        intersection = np.array([all_intersections[m,n] for m,n in assigned])
        union = np.array([(true_areas[m] + pred_areas[n] - all_intersections[m,n]) 
                           for m,n in assigned])
        iou = intersection / union

        # Remove background.
        iou = iou[1:]
        assigned = assigned[1:]
        true_labels = true_labels[1:]
        pred_labels = pred_labels[1:]

        # Labels that are not assigned.
        true_not_assigned = np.setdiff1d(true_labels, assigned[:,0])
        pred_not_assigned = np.setdiff1d(pred_labels, assigned[:,1])
        
    else:
        # in case that no object is identified in one of the masks
        iou = np.array([])
        assigned = np.array([])
        true_labels = true_labels[1:]
        pred_labels = pred_labels[1:]
        true_not_assigned = true_labels
        pred_not_assigned = pred_labels
        
    # Returning parameters.
    params = {'iou': iou, 'assigned': assigned, 'true_not_assigned': true_not_assigned,
             'pred_not_assigned': pred_not_assigned, 'true_labels': true_labels,
             'pred_labels': pred_labels}
    return params

def get_score_summary(y_true, y_pred):
    """Compute the score for a single sample including a detailed summary."""
    
    y_true_labeled = get_labeled_mask(y_true)  
    y_pred_labeled = get_labeled_mask(y_pred)  
    
    params = get_iou(y_true_labeled, y_pred_labeled)
    iou = params['iou']
    assigned = params['assigned']
    true_not_assigned = params['true_not_assigned']
    pred_not_assigned = params['pred_not_assigned']
    true_labels = params['true_labels']
    pred_labels = params['pred_labels']
    n_true_labels = len(true_labels)
    n_pred_labels = len(pred_labels)

    summary = []
    for i,threshold in enumerate(np.arange(0.5, 1.0, 0.05)):
        tp = np.sum(iou > threshold)
        fn = n_true_labels - tp
        fp = n_pred_labels - tp
        if (tp+fp+fn)>0: 
            prec = tp/(tp+fp+fn)
        else: 
            prec = 0
        summary.append([threshold, prec, tp, fp, fn])

    summary = np.array(summary)
    score = np.mean(summary[:,1]) # Final score.
    params_dict = {'summary': summary, 'iou': iou, 'assigned': assigned, 
                   'true_not_assigned': true_not_assigned, 
                   'pred_not_assigned': pred_not_assigned, 'true_labels': true_labels,
                   'pred_labels': pred_labels, 'y_true_labeled': y_true_labeled,
                   'y_pred_labeled': y_pred_labeled}
    
    return score, params_dict

def get_score(y_true, y_pred):
    """Compute the score for a batch of samples."""
    scores = []
    for i in range(len(y_true)):
        score,_ = get_score_summary(y_true[i],y_pred[i])
        scores.append(score)
    return np.array(scores)

def plot_score_summary(y_true, y_pred):
    """Plot score summary for a single sample."""
    # Compute score and assign parameters.
    score, params_dict = get_score_summary(y_true, y_pred)
    
    assigned = params_dict['assigned']
    true_not_assigned = params_dict['true_not_assigned']
    pred_not_assigned = params_dict['pred_not_assigned']
    true_labels = params_dict['true_labels']
    pred_labels = params_dict['pred_labels']
    y_true_labeled = params_dict['y_true_labeled']
    y_pred_labeled = params_dict['y_pred_labeled']
    summary = params_dict['summary']

    n_assigned = len(assigned)
    n_true_not_assigned = len(true_not_assigned)
    n_pred_not_assigned = len(pred_not_assigned)
    n_true_labels = len(true_labels)
    n_pred_labels = len(pred_labels)

    # Summary dataframe.
    summary_df = pd.DataFrame(summary,columns=['threshold','precision','tp','fp','fn'])
    print('Final score:', score)
    print(summary_df)

    # Plots.
    fig, axs = plt.subplots(2,3,figsize=(20,13))

    # True mask with true objects.
    img = y_true
    axs[0,0].imshow(img, cmap='gray')
    #axs[0,0].set_title('{}.) true mask: {} true objects'.format(n,train_df['num_masks'][n]))

    # True mask with identified objects.
    #img = np.zeros(y_true.shape)
    #img[y_true_labeled > 0.5] = 255
    img, img_type = imshow_args(y_true_labeled)
    axs[0,1].imshow(img, img_type)
    #axs[0,1].set_title('{}.) true mask: {} objects identified'.format(n, n_true_labels))

    # Predicted mask with identified objects.
    #img = np.zeros(y_true.shape)
    #img[y_pred_labeled > 0.5] = 255
    img, img_type = imshow_args(y_pred_labeled)
    axs[0,2].imshow(img, img_type)
    axs[0,2].set_title('{}.) predicted mask: {} objects identified'.format(
        n, n_pred_labels))

    # Prediction overlap with true mask.
    img = np.zeros(y_true.shape)
    img[y_true > 0.5] = 100
    for i,j in assigned: img[(y_true_labeled == i) & (y_pred_labeled == j)] = 255
    axs[1,0].set_title('{}.) {} pred. overlaps (white) with true objects (gray)'.format(
        n,len(assigned)))
    axs[1,0].imshow(img, cmap='gray', norm=None)

    # Intersection over union.
    img = np.zeros(y_true.shape)
    img[(y_pred_labeled > 0) & (y_pred_labeled < 100)] = 100
    img[(y_true_labeled > 0) & (y_true_labeled < 100)] = 100
    for i,j in assigned: img[(y_true_labeled == i) & (y_pred_labeled == j)] = 255
    axs[1,1].set_title('{}.) {} intersections (white) over unions (gray)'.format(
        n, n_assigned))
    axs[1,1].imshow(img, cmap='gray');

    # False positives and false negatives.
    img = np.zeros(y_true.shape)
    for i in pred_not_assigned: img[(y_pred_labeled == i)] = 255
    for i in true_not_assigned: img[(y_true_labeled == i)] = 100
    axs[1,2].set_title('{}.) no threshold: {} fp (white), {} fn (gray)'.format(
        n, n_pred_not_assigned, n_true_not_assigned))
    axs[1,2].imshow(img, cmap='gray');

# VISUALISING RESULTS

In [ ]:
n=0

testImg = x2[n]
testLab = u2[n].numpy()

# Feed-forward 
segImg = net(Variable(testImg).unsqueeze(0).cuda())
# Applying softmax to get class probabilities
segImg_np = F.softmax(segImg).data.cpu().squeeze(0).numpy()
segImg_np = (segImg_np>0.5)
segImg_np = skimage.morphology.label(segImg_np)

k2=testLab[0,:,:]
k=segImg_np[0,:,:]

plot_score_summary(k2,k)

# Displaying segmented output and ground truth
plt.figure(figsize=(15,15))
plt.subplot(231)
plt.imshow(segImg_np[0,:,:],cmap='gray')
plt.title('Channel 1')
plt.subplot(232)
plt.imshow(segImg_np[1,:,:],cmap='gray')
plt.title('Channel 2')
plt.subplot(233)
plt.imshow(segImg_np[2,:,:],cmap='gray')
plt.title('Channel 3')
plt.subplot(234)
plt.imshow(testLab[0,:,:],cmap='gray')
plt.title('Channel 1')
plt.subplot(235)
plt.imshow(testLab[1,:,:],cmap='gray')
plt.title('Channel 2')
plt.subplot(236)
plt.imshow(testLab[2,:,:],cmap='gray')
plt.title('Channel 3')
'''mean=torch.mean(torch.mean(testImg, dim=2),dim=2)
    for m in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        s=torch.std(testImg[m][j],unbiased=False)
        testImg[m][j]=(testImg[m][j]-mean[m][j])/s'''

# CREATING PTH FILE OF TEST IMAGES

In [ ]:
#create test dataset form images
parentlabelpath = 'F:\\study material\\DL\\MICCAI2\\test-images\\'
n=18 #no. of test images
labeltensor=torch.FloatTensor(n, 3, 256, 256)
for i in range(n):
    img1 = cv2.imread(parentlabelpath + 'image ('+ str(i+1) + ').png')#load image
    img1 = cv2.resize(img1,(256,256))
    label=np.array(img1)
    label=np.transpose(label, (2,0,1))
    label = torch.from_numpy(label)
    labeltensor[i] = label
torch.save(labeltensor, parentlabelpath + 'filename.pth')

In [ ]:
# load own testset
x3=torch.load("filename.pth")

# VISUALISE RESULT ON TEST IMAGE

In [ ]:
n=0 #sample no.

testImg = x3[n]
# Feed-forward 
segImg = net(Variable(testImg).unsqueeze(0).cuda())
# Applying softmax to get class probabilities
segImg_np = F.softmax(segImg).data.cpu().squeeze(0).numpy()
segImg_np = (segImg_np>0.5)
segImg_np = skimage.morphology.label(segImg_np)
# Displaying segmented output
plt.figure(figsize=(15,15))
plt.subplot(231)
plt.imshow(segImg_np[0,:,:],cmap='gray')
plt.title('Channel 1')
plt.subplot(232)
plt.imshow(segImg_np[1,:,:],cmap='gray')
plt.title('Channel 2')
plt.subplot(233)
plt.imshow(segImg_np[2,:,:],cmap='gray')
plt.title('Channel 3')
plt.subplot(234)